In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Keep classes 0,1,2 only
mask_train = (y_train < 3).flatten()
mask_test = (y_test < 3).flatten()
x_train = x_train[mask_train].astype("float32")/255.0
y_train = y_train[mask_train].flatten()
x_test = x_test[mask_test].astype("float32")/255.0
y_test = y_test[mask_test].flatten()

# Small CNN model
model = models.Sequential([
layers.Conv2D(32, (3,3), activation="relu", padding="same", input_shape=(32,32,3)),
layers.MaxPooling2D(),
layers.Conv2D(64, (3,3), activation="relu", padding="same"),
layers.MaxPooling2D(),
layers.Conv2D(128, (3,3), activation="relu", padding="same"),
layers.MaxPooling2D(),
layers.Flatten(),
layers.Dense(64, activation="relu"),
layers.Dense(3, activation="softmax")
])

model.compile(
optimizer="adam",
loss="sparse_categorical_crossentropy",
metrics=["accuracy"]
)

# Train
history = model.fit(
x_train, y_train,
epochs=5,
batch_size=64,
validation_split=0.1,
verbose=1
)

# Evaluate
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print("Test accuracy (tiny CNN):", acc)

C:\Users\rolan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.7221 - loss: 0.6502 - val_accuracy: 0.8313 - val_loss: 0.4549
Epoch 2/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.8330 - loss: 0.4179 - val_accuracy: 0.8527 - val_loss: 0.4019
Epoch 3/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.8665 - loss: 0.3458 - val_accuracy: 0.8873 - val_loss: 0.3189
Epoch 4/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.8859 - loss: 0.2900 - val_accuracy: 0.8933 - val_loss: 0.2898
Epoch 5/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.9019 - loss: 0.2573 - val_accuracy: 0.8720 - val_loss: 0.3411
Test accuracy (tiny CNN): 0.8569999933242798


BatchNorm + Dropout + EarlyStopping

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

mask_train = (y_train < 3).flatten()
mask_test  = (y_test < 3).flatten()

x_train = x_train[mask_train].astype("float32") / 255.0
y_train = y_train[mask_train].flatten()
x_test  = x_test[mask_test].astype("float32") / 255.0
y_test  = y_test[mask_test].flatten()

model = models.Sequential([
    layers.Input(shape=(32,32,3)),

    layers.Conv2D(32, 3, padding="same"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(32, 3, padding="same"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Conv2D(64, 3, padding="same"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, 3, padding="same"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Conv2D(128, 3, padding="same"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(3, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True
)

history = model.fit(
    x_train, y_train,
    epochs=30,
    batch_size=64,
    validation_split=0.1,
    callbacks=[cb],
    verbose=1
)

loss, acc = model.evaluate(x_test, y_test, verbose=0)
print("Test accuracy (upgraded CNN):", acc)


Epoch 1/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 31s 131ms/step - accuracy: 0.7138 - loss: 0.7287 - val_accuracy: 0.3533 - val_loss: 2.9864
Epoch 2/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 28s 131ms/step - accuracy: 0.8089 - loss: 0.4867 - val_accuracy: 0.4953 - val_loss: 1.6038
Epoch 3/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 27s 126ms/step - accuracy: 0.8394 - loss: 0.4142 - val_accuracy: 0.7980 - val_loss: 0.4941
Epoch 4/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 26s 125ms/step - accuracy: 0.8575 - loss: 0.3664 - val_accuracy: 0.7733 - val_loss: 0.5582
Epoch 5/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 31s 145ms/step - accuracy: 0.8841 - loss: 0.3202 - val_accuracy: 0.8867 - val_loss: 0.2959
Epoch 6/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 29s 137ms/step - accuracy: 0.8908 - loss: 0.2911 - val_accuracy: 0.8687 - val_loss: 0.3382
Epoch 7/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 29s 136ms/step - accuracy: 0.8986 - loss: 0.2659 - val_accuracy: 0.9040 - val_loss: 0.2630
Epoch 8/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 27s 127ms/step - accuracy: 0.9053 - loss: 0

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

mask_train = (y_train < 3).flatten()
mask_test  = (y_test < 3).flatten()

x_train = x_train[mask_train].astype("float32") / 255.0
y_train = y_train[mask_train].flatten()
x_test  = x_test[mask_test].astype("float32") / 255.0
y_test  = y_test[mask_test].flatten()

model = models.Sequential([
    layers.Input(shape=(32,32,3)),

    layers.Conv2D(32, 3, padding="same"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(32, 3, padding="same"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Conv2D(64, 3, padding="same"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, 3, padding="same"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Conv2D(128, 3, padding="same"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(3, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True
)

data_aug = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomZoom(0.1),
])

history = model.fit(
    x_train, y_train,
    epochs=30,
    batch_size=64,
    validation_split=0.1,
    callbacks=[cb],
    verbose=1
)

loss, acc = model.evaluate(x_test, y_test, verbose=0)
print("Test accuracy (upgraded CNN):", acc)


Transfer Learning (MobileNetV2)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

mask_train = (y_train < 3).flatten()
mask_test  = (y_test < 3).flatten()

x_train = x_train[mask_train].astype("float32")
y_train = y_train[mask_train].flatten()
x_test  = x_test[mask_test].astype("float32")
y_test  = y_test[mask_test].flatten()

IMG_SIZE = 96

preprocess = tf.keras.Sequential([
    layers.Resizing(IMG_SIZE, IMG_SIZE),
    layers.Lambda(tf.keras.applications.mobilenet_v2.preprocess_input)
])

base = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)
base.trainable = False

model = tf.keras.Sequential([
    layers.Input(shape=(32,32,3)),
    preprocess,
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(3, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=2, restore_best_weights=True
)

history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.1,
    callbacks=[cb],
    verbose=1
)

loss, acc = model.evaluate(x_test, y_test, verbose=0)
print("Test accuracy (transfer learning):", acc)
